In [1]:
import pandas as pd
import json

# 1. Укажите свои пути
csv_path  = '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings/decoded_queries_Chunking_Evaluation .csv'
file_path = '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings/court_decisions_combined.txt'

# 2. Читаем CSV и текст корпуса
df = pd.read_csv(csv_path)
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# 3. Список для несоответствий
mismatches = []

# 4. Проходим по каждой строке и проверяем все references
for idx, row in df.iterrows():
    # references — JSON-строка вида [{"content": "...", "start_index": X, "end_index": Y}, ...]
    try:
        refs = json.loads(row['references'])
    except json.JSONDecodeError:
        mismatches.append((idx, 'Invalid JSON in references'))
        continue

    for ref in refs:
        start   = ref.get('start_index')
        end     = ref.get('end_index')
        content = ref.get('content')

        # Проверяем наличие всех полей
        if start is None or end is None or content is None:
            mismatches.append((idx, f'Missing fields in reference: {ref}'))
            continue

        # Проверка выхода за границы
        if not (0 <= start < end <= len(text)):
            mismatches.append((idx, f'Index out of range: {start}-{end}'))
            continue

        # Извлекаем сниппет и сравниваем
        snippet = text[start:end]
        if snippet != content:
            mismatches.append((
                idx,
                f'Mismatch at {start}-{end}:\n'
                f'  CSV content:  {content[:50]}...\n'
                f'  File snippet: {snippet[:50]}...'
            ))

# 5. Вывод результатов
if mismatches:
    print(f"Найдено {len(mismatches)} несоответствий. Первые 10:")
    for i, msg in mismatches[:10]:
        print(f"Row {i}: {msg}")
else:
    print("Все отрывки совпадают с содержимым файла.")

Все отрывки совпадают с содержимым файла.
